In [1]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn import model_selection
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
train_df=pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')
test_df=pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')
submission_df=pd.read_csv('/kaggle/input/30-days-of-ml/sample_submission.csv')

In [3]:
train_df.head()


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [4]:
test_df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [5]:
input_cols =list(train_df.columns)[1:-1]
target_cols='target'
inputs_df = train_df[input_cols].copy()
targets = train_df[target_cols]



# Extracting Numeric and Categorical cols from data
numeric_cols=inputs_df.select_dtypes(exclude=['object']).columns.tolist()
categorical_cols = inputs_df.select_dtypes('object').columns.tolist()
# Splitting data into training and validation
train_inputs, val_inputs, train_targets, val_targets =train_test_split(inputs_df,targets,train_size=0.8,test_size=0.2,random_state=0)


my_cols = categorical_cols + numeric_cols
train_inputs=train_inputs[my_cols]
val_inputs=val_inputs[my_cols]
test_df=test_df[my_cols]

A simple XGBoost model

In [6]:
'''lr_tries = [0.0001, 0.001, 0.01, 0.2, 0.3]
def operate(n_est, lr):
    model=XGBRegressor(n_estimators = n_est, learning_rate = lr_tries[lr])
    # Preprocessing for numerical data
    numerical_transformer = StandardScaler()

    # Preprocessing for categorical data
    categorical_transformer = OrdinalEncoder()

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numeric_cols),('cat', categorical_transformer, categorical_cols)])

    from sklearn.pipeline import Pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

    # Preprocessing of training data, fit model 
    my_pipeline.fit(train_inputs, train_targets)

    # Preprocessing of validation data, get predictions
    preds = my_pipeline.predict(val_inputs)

    # Evaluate the model
    score = mean_squared_error(preds,val_targets,squared=False)
    return score

def main(args):
	low = 1
	high = 5000
	while low < high:
		mid = (low + high) // 2
		mid1 = mid + 1
		print(low, mid, mid1, high, flush = True)
		if (operate(mid, args) < operate(mid+1, args)):
			high = mid
		else:
			low = mid + 1
		
	print(low)
	
	return low
 
ideal = main(3)
print(ideal)
'''
lr_tries = [0.0001, 0.001, 0.01, 0.2, 0.3]
model=XGBRegressor(n_estimators = 200, learning_rate = 0.1)
# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OrdinalEncoder()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numeric_cols),('cat', categorical_transformer, categorical_cols)])

from sklearn.pipeline import Pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

# Preprocessing of training data, fit model 
my_pipeline.fit(train_inputs, train_targets)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(val_inputs)

# Evaluate the model
score = mean_squared_error(preds,val_targets,squared=False)
'''
for i in range(50):
    
    model=XGBRegressor(n_estimators = i * 10 + 10, learning_rate = 0.1)
    # Preprocessing for numerical data
    numerical_transformer = StandardScaler()

    # Preprocessing for categorical data
    categorical_transformer = OrdinalEncoder()

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numeric_cols),('cat', categorical_transformer, categorical_cols)])

    from sklearn.pipeline import Pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

    # Preprocessing of training data, fit model 
    my_pipeline.fit(train_inputs, train_targets)

    # Preprocessing of validation data, get predictions
    preds = my_pipeline.predict(val_inputs)

    # Evaluate the model
    score = mean_squared_error(preds,val_targets,squared=False)
    print(score)
'''


"\nfor i in range(50):\n    \n    model=XGBRegressor(n_estimators = i * 10 + 10, learning_rate = 0.1)\n    # Preprocessing for numerical data\n    numerical_transformer = StandardScaler()\n\n    # Preprocessing for categorical data\n    categorical_transformer = OrdinalEncoder()\n\n    # Bundle preprocessing for numerical and categorical data\n    preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numeric_cols),('cat', categorical_transformer, categorical_cols)])\n\n    from sklearn.pipeline import Pipeline\n    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])\n\n    # Preprocessing of training data, fit model \n    my_pipeline.fit(train_inputs, train_targets)\n\n    # Preprocessing of validation data, get predictions\n    preds = my_pipeline.predict(val_inputs)\n\n    # Evaluate the model\n    score = mean_squared_error(preds,val_targets,squared=False)\n    print(score)\n"

In [7]:
test_preds=my_pipeline.predict(test_df)

In [8]:
submission_df['target'] = test_preds
submission_df.to_csv('submission.csv', index=False)

Endregion XGBoost